In [18]:
import numpy as np
import scipy.sparse as sp
import torch
    

In [19]:
def accuracy(output, labels):
    # Find manimum element of each 
    preds = output.max(1)[1].type_as(labels)
    
    correct = preds.eq(labels).double()
    correct = correct.sum()
    return correct / len(labels)

def encode_onehot(labels):
    classes = set(labels)
    classes_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot

def sparse_mx_to_torch_sparse_tensor(sparse_mx):
    """Convert a scipy sparse matrix to a torch sparse tensor."""
    sparse_mx = sparse_mx.tocoo().astype(np.float32)
    indices = torch.from_numpy(
        np.vstack((sparse_mx.row, sparse_mx.col)).astype(np.int64))
    values = torch.from_numpy(sparse_mx.data)
    shape = torch.Size(sparse_mx.shape)
    return torch.sparse.FloatTensor(indices, values, shape)

def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

In [20]:
def load_data(path="./data/cora/", dataset="cora"):
    """Load citation network dataset (cora only for now)"""
    print('Loading {} dataset...'.format(dataset))

    idx_features_labels = np.genfromtxt("{}{}.content".format(path, dataset),
                                        dtype=np.dtype(str))
    features = sp.csr_matrix(idx_features_labels[:, 1:-1], dtype=np.float32)
    labels = encode_onehot(idx_features_labels[:, -1])

    # build graph
    idx = np.array(idx_features_labels[:, 0], dtype=np.int32)
    idx_map = {j: i for i, j in enumerate(idx)}
    edges_unordered = np.genfromtxt("{}{}.cites".format(path, dataset),
                                    dtype=np.int32)
    edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),
                     dtype=np.int32).reshape(edges_unordered.shape)
    adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                        shape=(labels.shape[0], labels.shape[0]),
                        dtype=np.float32)

    # build symmetric adjacency matrix
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)

    features = normalize(features)
    adj = normalize(adj + sp.eye(adj.shape[0]))

    idx_train = range(140)
    idx_val = range(200, 500)
    idx_test = range(500, 1500)

    features = torch.FloatTensor(np.array(features.todense()))
    labels = torch.LongTensor(np.where(labels)[1])
    adj = sparse_mx_to_torch_sparse_tensor(adj)

    idx_train = torch.LongTensor(idx_train)
    idx_val = torch.LongTensor(idx_val)
    idx_test = torch.LongTensor(idx_test)

    return adj, features, labels, idx_train, idx_val, idx_test

In [21]:
import math
import torch.nn as nn
import torch
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module

class SGC(nn.Module):
    def __init__(self, k, nfeat, nclass):
        super(SGC, self).__init__()
        self.k = k
        self.W = nn.Linear(nfeat, nclass)


    def forward(self, x, adj):
        k = self.k
        for i in range(k):
            x = torch.spmm(adj, x)
        x = self.W(x)
        return x

In [22]:
adj, features, labels, idx_train, idx_val, idx_test = load_data()

# Parameters currently hard coded 
epochs = 200
k = 2
lr = 0.2
wd = 0.000001




# Model and optmizer

model = SGC(k, features.size(1), labels.max().item()+1)
optimizer = torch.optim.Adam(model.parameters(), lr=lr,
                           weight_decay=wd)

Loading cora dataset...


In [23]:
# Training

for epoch in range(epochs):
	model.train()
	optimizer.zero_grad()
	output = model(features, adj)
	loss_train = torch.nn.functional.cross_entropy(output[idx_train], labels[idx_train])
	loss_train.backward()
	optimizer.step()
	if epoch % 10 == 0:
		with torch.no_grad():
			model.eval()
			output = model(features, adj)
			acc_val = accuracy(output[idx_val], labels[idx_val])
			print("Epoch:", epoch, "Loss:", loss_train.item(), "Validation Acc",format(acc_val))

model.eval()
output = model(features, adj)
acc_val = accuracy(output[idx_test], labels[idx_test])
print("Test Acc:", format(acc_val))



Epoch: 0 Loss: 1.9456653594970703 Validation Acc 0.49333333333333335
Epoch: 10 Loss: 0.8221799731254578 Validation Acc 0.7933333333333333
Epoch: 20 Loss: 0.3835243582725525 Validation Acc 0.83
Epoch: 30 Loss: 0.2209903746843338 Validation Acc 0.8233333333333334
Epoch: 40 Loss: 0.15177805721759796 Validation Acc 0.8233333333333334
Epoch: 50 Loss: 0.11864215135574341 Validation Acc 0.8166666666666667
Epoch: 60 Loss: 0.10149743407964706 Validation Acc 0.81
Epoch: 70 Loss: 0.09164921194314957 Validation Acc 0.8166666666666667
Epoch: 80 Loss: 0.08530830591917038 Validation Acc 0.8166666666666667
Epoch: 90 Loss: 0.08068900555372238 Validation Acc 0.8133333333333334
Epoch: 100 Loss: 0.07700848579406738 Validation Acc 0.81
Epoch: 110 Loss: 0.0739065557718277 Validation Acc 0.8066666666666666
Epoch: 120 Loss: 0.07122986763715744 Validation Acc 0.8033333333333333
Epoch: 130 Loss: 0.0688970685005188 Validation Acc 0.8033333333333333
Epoch: 140 Loss: 0.06685230880975723 Validation Acc 0.8066666666